In [42]:
import folium
import geopandas as gpd
import pandas as pd
# pd.set_option('display.max_rows', None)

In [43]:
# Import the shapefile with data
gdf = gpd.read_file('data/Census_Tracts_2020_Redistricting_Data.shp')
# Import labels given to tracts for district options and analysis
labels = pd.read_csv('data/tract_labels.csv').drop(['TRACT_NO', 'FIPS2020'], axis=1)
# Cast types to enable merge
gdf['STATE'] = gdf['STATE'].astype('int64')
gdf['COUNTY'] = gdf['COUNTY'].astype('int64')
gdf['TRACT'] = gdf['TRACT'].astype('int64')
# Merge the shapefile with the labels
gdf = gdf.merge(labels, on=['STATE', 'COUNTY', 'TRACT'], how='left')
gdf = gdf[gdf['LABEL']!='Not Portland']
gdf = gdf[gdf['LABEL'].notnull()]

Geopandas helper maps

In [4]:
# # Use the Geopandas Choropleth maps as a guide by tract + population

# # Population
# ax = gdf.plot(figsize=(100, 100), 
#                  column='P2_001N', #'P2_001N', 
#                  legend=False,
#                  cmap='spring', # https://matplotlib.org/2.0.2/users/colormaps.html
#                  edgecolor='black', 
#                  linewidth=3,)
# gdf.apply(lambda x: ax.annotate(text=x['P2_001N'], 
#                                    xy=x.geometry.centroid.coords[0], 
#                                    ha='center',
#                                    size=11),
#              axis=1)
# # ax.set_title('Total Coronavirus Deaths as of September 2021\n', size=25)
# ax.set_xticks([])
# ax.set_yticks([])

In [5]:
# # Use the Geopandas Choropleth maps as a guide by tract + population

# # Census Tract
# ax = gdf.plot(figsize=(50, 50), 
#                  column='P2_001N', 
#                  legend=False,
#                  cmap='spring', # https://matplotlib.org/2.0.2/users/colormaps.html
#                  edgecolor='black', 
#                  linewidth=3,)
# gdf.apply(lambda x: ax.annotate(text=x['TRACT_NO'], 
#                                    xy=x.geometry.centroid.coords[0], 
#                                    ha='center',
#                                    size=11),
#              axis=1)
# # ax.set_title('Total Coronavirus Deaths as of September 2021\n', size=25)
# ax.set_xticks([])
# ax.set_yticks([])

Folium maps for some options

In [45]:
# Create a map and add choropleth by population for each tract
def map_districts(gdf, column, colors):
    """
    gfd is the geo data frame
    column is the name of the column in gdf used for coloring the choropleth map
    colors is the matplotlib cmap coloring scheme:
         https://matplotlib.org/2.0.2/users/colormaps.html
    """
    m = folium.Map(location=[45.5, -122.5], 
                   zoom_start=10, 
                   width='100%')
    cp = folium.Choropleth(
        geo_data=gdf,
        name='Portland, Oregon',
        data=gdf,
        columns=['TRACT_NO', column],
        key_on='feature.properties.TRACT_NO',
        fill_color=colors, 
        label=['P2_001N'],
        tooltip=['TRACT_NO', 'P2_001N'],
        fill_opacity=0.5,
        line_opacity=0.9,
#         legend_name='Portland Potential Districts',
    ).add_to(m)

    # This adds tooltips:  https://medium.com/analytics-vidhya/interactive-choropleth-map-in-python-using-folium-4e1479d9e568
    style_function = lambda x: {'fillColor': '#ffffff', 
                                'color':'#000000', 
                                'fillOpacity': 0.1, 
                                'weight': 0.1}
    tooltipadd = folium.features.GeoJson(
        gdf,
        style_function=style_function, 
        control=False,
        tooltip=folium.features.GeoJsonTooltip(
            fields=['TRACT_NO', 'P2_001N'],
            aliases=['TRACT_NO', 'P2_001N'],
            style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
        )
    )
    m.add_child(tooltipadd)
    m.keep_in_front(tooltipadd)
    folium.LayerControl().add_to(m)
    return m

Folium map of Portland "quadrants" by population tract
* *census tracts do not adhere strictly to the quadrants, therefore this is an estimate*

In [49]:
# Create a map and add choropleth for Portland's population by quadrants
quadrants = gdf[gdf['LABEL'].notnull()]
quadrants['LABEL_NUM'] = quadrants['LABEL'].replace(
                          {'NE': 1, 'SW': 2, 'NW': 3, 'SE': 4, 'N': 5}).astype('float64')
quadrants_map = map_districts(quadrants, 'LABEL_NUM', 'Accent')
# quadrants_map

Validations 

In [33]:
# # The data is correctly labeled for quadrants
# print(gdf['LABEL'].unique())
# gdf_north = gdf[gdf['LABEL']=='N']
# map_districts(gdf_north, 'P2_001N', 'OrRd')

In [39]:
# The data is correctly labeled for neighborhood associations
print(gdf['NEIGHBORHOOD_ASSOCIATION'].unique())
gdf_NA = gdf.copy()
gdf_NA['NA_NUM'] = gdf_NA['NEIGHBORHOOD_ASSOCIATION'].replace(
    {'NECN': 1, 'SWNI': 2, 'NWNW': 3, 'EPNO': 4, 'NPNS': 5, 'SEUL': 6,
     'CNN': 7, 'LLOYD': 8}).astype('float64')
# map_districts(gdf_NA, 'NA_NUM', 'Accent')

['NECN' 'SWNI' 'NWNW' 'EPNO' 'NPNS' 'SEUL' 'CNN' 'LLOYD']


In [50]:
# Option 1 - Trying to keep the neighborhood associations whole, not 100% possible
option_1 = gdf[gdf['OPTION_1'].notnull()]
print(len(option_1))
option_1_map = map_districts(option_1, 'OPTION_1', 'Accent')
# option_1_map
# m.save('option_1.html')

162
